In [1]:
import pandas as pd
import geopandas as gpd
import mio
import pathlib
from matplotlib import pyplot as plt
import numpy as np

In [2]:
k25 = pd.read_excel('komb25.xlsx')
k25.head(3)

,map_name,xmin,xmax,ymin,ymax
0,Komb1011,672500,690000,290000,302000
1,Komb1012,690000,707500,290000,302000
2,Komb1031,672500,690000,278000,290000


In [3]:
def make_sl_bui(tag):
    s = k25[k25.map_name == f'Komb{tag}'].iloc[0]
    xmin, ymin, xmax, ymax = s.xmin, s.ymin, s.xmax, s.ymax
    empty = pd.DataFrame(
            columns = range(xmin, xmax, 1),
            index = range(ymax, ymin, -1),
            data = 0
    )
    tifs = list(pathlib.Path('../sb_tif').glob(f'{tag}*.tif'))
    res = empty.copy()
    for tif in tifs:
        print(pathlib.Path(tif).name, end=' ')
        r = (
            mio.read_raster(tif)[0]
            .reindex( columns = range(xmin, xmax, 1),
                     index = range(ymax, ymin, -1),
                     fill_value = 0
                    )
        )
        res = np.maximum(res, r)
    # uint16 becoause above sea level !!
    mio.write_raster(res.astype('uint16'), f'../25k_sl_bui/{tag}_sl_bui.bil')

In [4]:
%%time
make_sl_bui('1091')

1091-11_raster.tif 1091-12_raster.tif 1091-13_raster.tif 1091-14_raster.tif 1091-21_raster.tif 1091-22_raster.tif 1091-23_raster.tif 1091-24_raster.tif 1091-31_raster.tif 1091-32_raster.tif 1091-33_raster.tif 1091-34_raster.tif 1091-41_raster.tif 1091-42_raster.tif 1091-43_raster.tif 1091-44_raster.tif Wall time: 1min 19s
